# Named Entity Recognition for Russian

<div class="admonition note" name="html-admonition" style="background: lightblue; padding: 10px">
<p class="title">Note</p>
This section, "Working in Languages Beyond English," is co-authored with <a href="http://www.quinndombrowski.com/">Quinn Dombrowski</a>, the Academic Technology Specialist at Stanford University and a leading voice in multilingual digital humanities. I'm grateful to Quinn for helping expand this textbook to serve languages beyond English. 
</div>

In this lesson, we're going to learn about a text analysis method called *Named Entity Recognition* (NER) as applied to Russian. This method will help us computationally identify people, places, and things (of various kinds) in a text or collection of texts.

---

## Dataset

The example text for Russian is *Яблони цветут* from *Новые люди* by Зинаида Николаевна Гиппиус [from Библиотека русской и советской классики](https://ruslit.traumlibrary.net/book/gippius-ss15-01/gippius-ss15-01.html). (Thanks to Katherine Bowers for the text referral.)

**Here's a preview of spaC's NER tagging *Яблони цветут*.**

If you compare the results to the [English example](Named-Entity-Recognition), you'll notice that the Russian NER is much less good at recognizing entities, and is especially bad ata distinguishing different kinds of entities, like ORG vs LOC. You need a lot of examples to train a model to distinguish different entity types; currently, English is the only model that does a decent job of it.

You can read more about the [data sources used to train Russian](https://spacy.io/models/ru) on the spaCy model page.

In [14]:
displacy.render(document, style="ent")

---

## NER with spaCy
If you've already used the pre-processing notebook for this language, you can skip the steps for installing spaCy and downloading the language model.

### Install spaCy
Russian models are only available starting in spaCy 3.0. 

If you run into errors because spaCy 2.x is installed, you can run `!pip uninstall spacy -y` first, then run the cell below.

In [ ]:
!pip install -U spacy

### Import Libraries

We're going to import `spacy` and `displacy`, a special spaCy module for visualization.

In [1]:
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
pd.options.display.max_rows = 600
pd.options.display.max_colwidth = 400

We're also going to import the `Counter` module for counting people, places, and things, and the `pandas` library for organizing and displaying data (we're also changing the pandas default max row and column width display setting).

### Download Language Model

Next we need to download the Russian-language model (`ru_core_news_md`), which will be processing and making predictions about our texts. You can read more about the [data sources used to train Russian](https://spacy.io/models/ru) on the spaCy model page.

In [2]:
!python -m spacy download ru_core_news_md

     ---------------------------------------- 0.0/41.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/41.9 MB 4.3 MB/s eta 0:00:10
     ---------------------------------------- 0.4/41.9 MB 4.9 MB/s eta 0:00:09
      --------------------------------------- 0.7/41.9 MB 5.5 MB/s eta 0:00:08
      --------------------------------------- 0.9/41.9 MB 5.4 MB/s eta 0:00:08
     - -------------------------------------- 1.2/41.9 MB 5.4 MB/s eta 0:00:08
     - -------------------------------------- 1.5/41.9 MB 5.6 MB/s eta 0:00:08
     - -------------------------------------- 2.0/41.9 MB 6.3 MB/s eta 0:00:07
     -- ------------------------------------- 2.5/41.9 MB 7.0 MB/s eta 0:00:06
     --- ------------------------------------ 3.2/41.9 MB 7.7 MB/s eta 0:00:06
     --- ------------------------------------ 3.8/41.9 MB 8.3 MB/s eta 0:00:05
     --- ------------------------------------ 3.8/41.9 MB 7.9 MB/s eta 0:00:05
     --- ------------------------------------ 4.1/41.9 MB 7

### Load Language Model

Once the model is downloaded, we need to load it. There are two ways to load a spaCy language model.

**1.** We can import the model as a module and then load it from the module.

In [3]:
import ru_core_news_md
nlp = ru_core_news_md.load()

**2.** We can load the model by name.

In [ ]:
#nlp = spacy.load('ru_core_news_md')

If you just downloaded the model for the first time, it's advisable to use Option 1. Then you can use the model immediately. Otherwise, you'll likely need to restart your Jupyter kernel (which you can do by clicking Kernel -> Restart Kernel.. in the Jupyter Lab menu).

## Process Document

We first need to process our `document` with the loaded NLP model. Most of the heavy NLP lifting is done in this line of code.

After processing, the `document` object will contain tons of juicy language data — named entities, sentence boundaries, parts of speech — and the rest of our work will be devoted to accessing this information.

In the cell below, we open and the example document. Then we run`nlp()` on the text and create our document.

In [4]:
filepath = '../texts/ru.txt'
text = open(filepath, encoding='utf-8').read()
document = nlp(text)

## Get Named Entities

All the named entities in our `document` can be found in the `document.ents` property. If we check out `document.ents`, we can see all the entities from the example document.

In [5]:
document.ents

(Петербурге,
 Петербурге,
 Володя,
 Москву,
 Володя,
 Сыромятников,
 Маремьянов,
 Москву,
 экономка,
 Москве,
 Малой Никитской,
 Володя,
 Москве,
 Листьев,
 Коренева,
 Коренева,
 Кореневой,
 Марфа,
 Марта,
 Марта,
 Володя,
 Марта,
 Марты,
 Марта,
 Марта,
 Голос,
 Марта,
 Мартой,
 Марты,
 Володя,
 Марфу Кореневу,
 Марту,
 Марте,
 Марта,
 Володя,
 Марту,
 Марта,
 Марта,
 Марта,
 Марта,
 Небо,
 Марта,
 Марта,
 Марты,
 Марта,
 Пора,
 Марта,
 Москвы,
 Марте,
 Петербурге,
 Марты)

Each of the named entities in `document.ents` contains [more information about itself](https://spacy.io/usage/linguistic-features#accessing), which we can access by iterating through the `document.ents` with a simple `for` loop.

For each `named_entity` in `document.ents`, we will extract the `named_entity` and its corresponding `named_entity.label_`.

In [6]:
for named_entity in document.ents:
    print(named_entity, named_entity.label_)

Петербурге LOC
Петербурге LOC
Володя PER
Москву LOC
Володя PER
Сыромятников PER
Маремьянов PER
Москву LOC
экономка PER
Москве LOC
Малой Никитской LOC
Володя PER
Москве LOC
Листьев PER
Коренева PER
Коренева PER
Кореневой PER
Марфа PER
Марта PER
Марта PER
Володя PER
Марта PER
Марты PER
Марта PER
Марта PER
Голос PER
Марта PER
Мартой PER
Марты PER
Володя PER
Марфу Кореневу PER
Марту PER
Марте PER
Марта PER
Володя PER
Марту PER
Марта PER
Марта PER
Марта PER
Марта PER
Небо PER
Марта PER
Марта PER
Марты PER
Марта PER
Пора PER
Марта LOC
Москвы LOC
Марте PER
Петербурге LOC
Марты PER


To extract just the named entities that have been identified as `PER` (person), we can add a simple `if` statement into the mix:

In [7]:
for named_entity in document.ents:
    if named_entity.label_ == "PER":
        print(named_entity)

Володя
Володя
Сыромятников
Маремьянов
экономка
Володя
Листьев
Коренева
Коренева
Кореневой
Марфа
Марта
Марта
Володя
Марта
Марты
Марта
Марта
Голос
Марта
Мартой
Марты
Володя
Марфу Кореневу
Марту
Марте
Марта
Володя
Марту
Марта
Марта
Марта
Марта
Небо
Марта
Марта
Марты
Марта
Пора
Марте
Марты


## NER with Long Texts or Many Texts

In [8]:
import math
number_of_chunks = 80

chunk_size = math.ceil(len(text) / number_of_chunks)

text_chunks = []

for number in range(0, len(text), chunk_size):
    text_chunk = text[number:number+chunk_size]
    text_chunks.append(text_chunk)

In [9]:
chunked_documents = list(nlp.pipe(text_chunks))

## Get People

To extract and count the people, we will use an `if` statement that will pull out words only if their "ent" label matches "PER."

In [10]:
people = []

for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == "PER":
            people.append(named_entity.text)

people_tally = Counter(people)

df = pd.DataFrame(people_tally.most_common(), columns=['character', 'count'])
df

,character,count
0,Марта,13
1,Володя,6
2,Марты,4
3,Коренева,2
4,Марту,2
5,Марте,2
6,Сыромятников,1
7,Маремьянов,1
8,экономка,1
9,Листьев,1


## Get Places

To extract and count places, we can follow the same model as above, except we will change our `if` statement to check for "ent" labels that match "LOC."

In [11]:
places = []
for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == "LOC":
            places.append(named_entity.text)

places_tally = Counter(places)

df = pd.DataFrame(places_tally.most_common(), columns=['place', 'count'])
df

,place,count
0,Петербурге,3
1,Москву,2
2,Москве,2
3,Малой Никитской,1
4,Марта,1
5,Москвы,1


## Get NER in Context

In [12]:
from IPython.display import Markdown, display
import re

def get_ner_in_context(keyword, document, desired_ner_labels= False):
    
    if desired_ner_labels != False:
        desired_ner_labels = desired_ner_labels
    else:
        desired_ner_labels = ['PER', 'ORG', 'LOC']  
        
    #Iterate through all the sentences in the document and pull out the text of each sentence
    for sentence in document.sents:
        #process each sentence
        sentence_doc = nlp(sentence.text)
        for named_entity in sentence_doc.ents:
            #Check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
            if keyword.lower() in named_entity.text.lower()  and named_entity.label_ in desired_ner_labels:
                #Use the regex library to replace linebreaks and to make the keyword bolded, again ignoring capitalization
                #sentence_text = sentence.text
            
                sentence_text = re.sub('\n', ' ', sentence.text)
                sentence_text = re.sub(f"{named_entity.text}", f"**{named_entity.text}**", sentence_text, flags=re.IGNORECASE)

                display(Markdown('---'))
                display(Markdown(f"**{named_entity.label_}**"))
                display(Markdown(sentence_text))

In [13]:
for document in chunked_documents:
    get_ner_in_context('Марта', document)

---

**PER**

– Да, так вы удивились, что я **Марта**.

---

**PER**

...

**PER**

Когда прошло время, и все кругом нас стало яснее и холоднее, небо позеленело, и утренние сумерки спустились, – я взглянул близко в лицо Марты, **Марта** сидела все в том же положении, прижавшись ко мне.

---

**LOC**

Она осталась на скамейке и не смотрела на меня.  – Прощай, **Марта**, – сказал я.  – Прощай.